In [2]:
import numpy as np 
import torch 


In [13]:
a=np.load("../irl/expert_data/hopper_expert_states.npy")

In [3]:
# Imports
import gym
import torch.optim as optim
from utils import OptimizerSpec
import time
from deep_q_learner import deep_Q_learning
from utils import *

# Import Taxi environment
env = gym.make('Taxi-v3')

# Define all the model parameters
hidden_units = 32               # num. units in hidden layer
replay_buffer_size = 200000     # buffer size
start_learning = 50000          # num. transitions before start learning
target_update_freq = 10000      # num. transitions between Q_target network updates
eps = 0.1                       # final epsilon for epsilon-greedy action selection
schedule_timesteps = 350000     # num. transitions for epsilon annealing
batch_size = 32                 # size of batch size for training
gamma = 0.99                    # discount factor of MDP
eps_optim = 0.01                # epsilon parameter for optimization (improves stability of optimizer)
alpha = 0.95                    # alpha parameter of RMSprop optimizer
learning_rate = 0.00025         # step size for optimization process


encode_method = 'one_hot'     # state encoding method ('one_hot' or 'positions')

if encode_method is 'positions':
    state_dim = 19  # explained in utils.encode_states
else:   # one-hot
    state_dim = env.observation_space.n

regularization = None           # regularization may be 'regularization'
save_fig = True     # whether to save figure of accumulated reward
save_model = True      # whether to save the DQN model

# Define 2-layered architecture
architecture = {"state_dim": state_dim,
                "hidden_units": hidden_units,
                "num_actions": env.action_space.n}
# Pack the epsilon greedy exploration parameters
exploration_params = {"timesteps": schedule_timesteps, "final_eps": eps}
# Define optimization procedure
optimizer_spec = OptimizerSpec(
        constructor=optim.RMSprop,
        kwargs=dict(lr=learning_rate, alpha=alpha, eps=eps_optim),)
        # kwargs=dict(lr=learning_rate),)


<>:28: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
<>:28: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
/Users/ksmkumar/mohan/IRL/mujoco_erricson/Taxi/deep_q_learner.py:69: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
  if regularization is 'dropout':
/Users/ksmkumar/mohan/IRL/mujoco_erricson/Taxi/deep_q_learner.py:190: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
  if regularization is 'l1':
/var/folders/30/6_15k8xn4kn7vwrk6x8py9sc0000gq/T/ipykernel_6061/3283725553.py:28: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
  if encode_method is 'positions':


In [8]:
env = gym.make("Taxi-v3")

# Variables to store expert trajectories

# Set the maximum number of points to collect
num_points = 2000
model_path="./trained_DQN_model"
dqn=torch.load(model_path)


/var/folders/30/6_15k8xn4kn7vwrk6x8py9sc0000gq/T/ipykernel_6061/1501018712.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dqn=torch.load(model_path)


In [13]:
from utils import * 
state,_=env.reset()




array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [19]:
dqn.eval()
dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
def select_action(state, dqn):
    # Convert state to torch tensor
    state=encode_states([state],encode_method,state_dim)
    state = Variable(torch.from_numpy(state).type(dtype))
    return int(dqn(state).data.argmax())
select_action(state,dqn)

0

In [38]:
# Collect exactly 2000 points (steps)
steps_collected = 0
state,_= env.reset()
np.bool8=np.bool
states = []
actions = []
next_states = []
dones = []

while steps_collected < num_points:
    # Expert selects an action using DQN
  
    action = select_action(state,dqn)
    # state=encode_states([state],encode_method,state_dim)[0]
    # Perform action in the environment
    next_state, reward, done, _,_= env.step(action)
    
    # Store the expert trajectory
    states.append(state)
    actions.append(action)
    next_states.append(next_state)
    dones.append(done)
    
    # Move to the next state
    state = next_state
    steps_collected += 1

    # Reset environment if done
    if done:
        state,_ = env.reset()

# Convert lists to numpy arrays
states = np.array(states)
actions = np.array(actions)
next_states = np.array(next_states)
dones = np.array(dones)

# Directory to save the expert data
expert_data_dir = "expert_data/"
state_file = expert_data_dir + "taxi_expert_states"
action_file = expert_data_dir + "taxi_expert_actions.npy"
next_state_file = expert_data_dir + "taxi_expert_next_states.npy"
done_file = expert_data_dir + "taxi_expert_dones.npy"

# Save the expert trajectories to .npy files
np.save(state_file, states)
np.save(action_file, actions)
np.save(next_state_file, next_states)
np.save(done_file, dones)

print(f"Expert trajectories saved with exactly {num_points} points.")

Expert trajectories saved with exactly 2000 points.


In [34]:
print(actions.shape)

(2000,)
